# Day 15

Puzzle 1: There's some moveable boxes and immovable walls. There's a robot that tires to move and can move the boxes if it hits them. Calculate some movements. 

In [34]:
with open("inputs/input15") as file:
    lines = file.readlines()

lines = [line.strip() for line in lines]

rows = lines.index("")
cols = len(lines[0])

directions = "".join(lines[rows + 1 :])

walls = [[0] * cols for _ in range(rows)]
boxes = [[0] * cols for _ in range(rows)]

for i in range(rows):
    for j in range(cols):
        if lines[i][j] == "#":
            walls[i][j] = 1
        if lines[i][j] == "O":
            boxes[i][j] = 1
        if lines[i][j] == "@":
            at = (i, j)


def try_to_move(at, dir_tuple):
    i = at[0] + dir_tuple[0]
    j = at[1] + dir_tuple[1]

    if walls[i][j]:
        return  # can't move
    if boxes[i][j]:
        try_to_move((i, j), dir_tuple)

    # Now that we've dealt with what's in front:
    if boxes[i][j]:  # it's still there
        return
    # otherwise, there's space, move a box if it exists
    if boxes[at[0]][at[1]]:
        boxes[at[0]][at[1]] = 0
        boxes[i][j] = 1

def visualize(at, boxes, walls):
    viz = [["."] * cols for _ in range(rows)]
    for i in range(rows):
        for j in range(cols):
            if at == (i, j):
                viz[i][j] = "@"
            if walls[i][j]:
                viz[i][j] = "#"
            if boxes[i][j]:
                viz[i][j] = "O"
            if (((at == (i, j)) + walls[i][j] + boxes[i][j]) > 1):
                viz[i][j] = "X"  # invalid state.
    viz = ["".join(v) for v in viz]
    for v in viz:
        print(v)

# print("Initial state:")
# visualize(at, boxes, walls)
# print()
for dir in directions:
    match dir:
        case ">":
            dir_tuple = (0, 1)
        case "<":
            dir_tuple = (0, -1)
        case "^":
            dir_tuple = (-1, 0)
        case "v":
            dir_tuple = (1, 0)
    
    try_to_move(at, dir_tuple)
    i = at[0] + dir_tuple[0]
    j = at[1] + dir_tuple[1]
    if not walls[i][j] and not boxes[i][j]:
        at = (i, j)
    # print(f"Move {dir}:")
    # visualize(at, boxes, walls)
    # print()

total = 0
for i in range(rows):
    for j in range(cols):
        if boxes[i][j]:
            total += 100 * i + j

print(total)

1505963


Part 2: The boxes are actually 2-wide. Do the logic with this.

In [ ]:
with open("inputs/input15") as file:
    lines = file.readlines()

lines = [line.strip() for line in lines]

rows = lines.index("")
cols = len(lines[0])

directions = "".join(lines[rows + 1 :])

walls = [[0] * cols for _ in range(rows)]
boxes = [[0] * cols for _ in range(rows)]

for i in range(rows):
    for j in range(cols):
        if lines[i][j] == "#":
            walls[i][j] = 1
        if lines[i][j] == "O":
            boxes[i][j] = 1
        if lines[i][j] == "@":
            at = (i, j)


def try_to_move(at, dir_tuple):
    i = at[0] + dir_tuple[0]
    j = at[1] + dir_tuple[1]

    if walls[i][j]:
        return  # can't move
    if boxes[i][j]:
        try_to_move((i, j), dir_tuple)

    # Now that we've dealt with what's in front:
    if boxes[i][j]:  # it's still there
        return
    # otherwise, there's space, move a box if it exists
    if boxes[at[0]][at[1]]:
        boxes[at[0]][at[1]] = 0
        boxes[i][j] = 1

def visualize(at, boxes, walls):
    viz = [["."] * cols for _ in range(rows)]
    for i in range(rows):
        for j in range(cols):
            if at == (i, j):
                viz[i][j] = "@"
            if walls[i][j]:
                viz[i][j] = "#"
            if boxes[i][j]:
                viz[i][j] = "O"
            if (((at == (i, j)) + walls[i][j] + boxes[i][j]) > 1):
                viz[i][j] = "X"  # invalid state.
    viz = ["".join(v) for v in viz]
    for v in viz:
        print(v)

# print("Initial state:")
# visualize(at, boxes, walls)
# print()
for dir in directions:
    match dir:
        case ">":
            dir_tuple = (0, 1)
        case "<":
            dir_tuple = (0, -1)
        case "^":
            dir_tuple = (-1, 0)
        case "v":
            dir_tuple = (1, 0)
    
    try_to_move(at, dir_tuple)
    i = at[0] + dir_tuple[0]
    j = at[1] + dir_tuple[1]
    if not walls[i][j] and not boxes[i][j]:
        at = (i, j)
    # print(f"Move {dir}:")
    # visualize(at, boxes, walls)
    # print()

total = 0
for i in range(rows):
    for j in range(cols):
        if boxes[i][j]:
            total += 100 * i + j

print(total)

# Day 16

Puzzle 1: There's a maze, with a small penalty for stepping forward and a large penalty for rotating 90 degrees. What's the smallest possible penalty?

In [ ]:
from collections import defaultdict

with open('inputs/input16') as file:
    lines = file.readlines()

lines = [line.strip() for line in lines]
rows = len(lines)
cols = len(lines[0])
free = set()
for i in range(rows):
    for j in range(cols):
        match lines[i][j]:
            case ".":
                free.add((i, j))
            case "S":
                start = (i, j)
                free.add((i, j))
            case "E":
                end = (i, j)
                free.add((i, j))

def get_nbhs_costs(node):
    result = []
    at, facing = node
    match facing:
        case ">":
            dir_tuple = (0, 1)
        case "<":
            dir_tuple = (0, -1)
        case "^":
            dir_tuple = (-1, 0)
        case "v":
            dir_tuple = (1, 0)
    in_front = (at[0] + dir_tuple[0], at[1] + dir_tuple[1])
    if in_front in free:
        result.append(((in_front, facing), 1))
    if facing in "<>":
        result.append(((at, "^"), 1000))
        result.append(((at, "v"), 1000))
    if facing in "^v":
        result.append(((at, "<"), 1000))
        result.append(((at, ">"), 1000))

    return result
    
# implement Dijkstras

min_cost = defaultdict(lambda: float('inf'))
candidates = defaultdict(lambda: float('inf'))
candidates[(start, ">")] = 0
while candidates:
    # find min candidate
    best_score = float('inf')
    best_node = None
    for node, dist in candidates.items():
        if dist < best_score:
            best_score = dist
            best_node = node
    del candidates[best_node]
    min_cost[best_node] = best_score
    nbhs_costs = get_nbhs_costs(best_node)
    for node, cost in nbhs_costs:
        if node not in min_cost:
            candidates[node] = min(candidates[node], best_score + cost)

print(min(min_cost[(end, c)] for c in "<>^v"))



94444


In [65]:
# Failed attempt at part 1. Tried to do dynamic programming with recursion on it, 
# but the copmutation graph is not acyclic. So not sure how to do it.
from functools import cache

with open('inputs/input16') as file:
    lines = file.readlines()

lines = [line.strip() for line in lines]
rows = len(lines)
cols = len(lines[0])
free = set()
for i in range(rows):
    for j in range(cols):
        match lines[i][j]:
            case ".":
                free.add((i, j))
            case "S":
                start = (i, j)
                free.add((i, j))
            case "E":
                end = (i, j)
                free.add((i, j))

@cache
def optimal_path(at, facing, visited=None):
    if visited is None:
        visited = frozenset()
    if (at, facing) in visited:
        return float('inf')
    if at == end:
        return 0
    # forward
    match facing:
        case ">":
            dir_tuple = (0, 1)
        case "<":
            dir_tuple = (0, -1)
        case "^":
            dir_tuple = (-1, 0)
        case "v":
            dir_tuple = (1, 0)
    in_front = (at[0] + dir_tuple[0], at[1] + dir_tuple[1])
    forward_score = (optimal_path(in_front, facing, visited.union([(at, facing)])) + 1) if in_front in free else float('inf')
    
    if facing in "<>":
        turn_score_1 = optimal_path(at, "^", visited.union([(at, facing)])) + 1000
        turn_score_2 = optimal_path(at, "v", visited.union([(at, facing)])) + 1000
    if facing in "^v":
        turn_score_1 = optimal_path(at, "<", visited.union([(at, facing)])) + 1000
        turn_score_2 = optimal_path(at, ">", visited.union([(at, facing)])) + 1000

    return min(forward_score, turn_score_1, turn_score_2)

print(optimal_path(start, ">"))

def visualize():
    viz = [["#"] * cols for _ in range(rows)]
    for point in free:
        viz[point[0]][point[1]] = "."
    viz = ["".join(v) for v in viz]
    for v in viz:
        print(v)
    

frozenset({1})

Part 2: Now count the number of spaces which are part of any best path. 

In [39]:
from collections import defaultdict

with open('inputs/input16') as file:
    lines = file.readlines()

lines = [line.strip() for line in lines]
rows = len(lines)
cols = len(lines[0])
free = set()
for i in range(rows):
    for j in range(cols):
        match lines[i][j]:
            case ".":
                free.add((i, j))
            case "S":
                start = (i, j)
                free.add((i, j))
            case "E":
                end = (i, j)
                free.add((i, j))

def get_nbhs_costs(node):
    result = []
    at, facing = node
    match facing:
        case ">":
            dir_tuple = (0, 1)
        case "<":
            dir_tuple = (0, -1)
        case "^":
            dir_tuple = (-1, 0)
        case "v":
            dir_tuple = (1, 0)
    in_front = (at[0] + dir_tuple[0], at[1] + dir_tuple[1])
    behind = (at[0] - dir_tuple[0], at[1] - dir_tuple[1])
    if in_front in free:
        result.append(((in_front, facing), 1))
    if behind in free:
        result.append(((behind, facing), 1))
    if facing in "<>":
        result.append(((at, "^"), 1000))
        result.append(((at, "v"), 1000))
    if facing in "^v":
        result.append(((at, "<"), 1000))
        result.append(((at, ">"), 1000))

    return result
    
# implement Dijkstras

min_cost = defaultdict(lambda: float('inf'))
candidates = defaultdict(lambda: float('inf'))
candidates[(start, ">")] = 0
while candidates:
    # find min candidate
    best_score = float('inf')
    best_node = None
    for node, dist in candidates.items():
        if dist < best_score:
            best_score = dist
            best_node = node
    del candidates[best_node]
    min_cost[best_node] = best_score
    nbhs_costs = get_nbhs_costs(best_node)
    for node, cost in nbhs_costs:
        if node not in min_cost:
            candidates[node] = min(candidates[node], best_score + cost)

# Do this so we know the best path length instead of hardcoding the value
best_path_cost = min(min_cost[(end, c)] for c in "<>^v")

# Idea: Once we have best costs, for each node, we can find the nodes which are 
# part of the optimal path leading to that node. So every node has "parents"
# and we want to find the parents of the end node.count

parents = defaultdict(list)
# for node, cost in min_cost.items():
#     for maybe_nbh in min_cost.keys():
#         for nbh_nbh, nbh_nbh_cost in get_nbhs_costs(maybe_nbh):
#             if nbh_nbh == node and min_cost[node] == min_cost[maybe_nbh] + nbh_nbh_cost:
#                 parents[node].append(maybe_nbh)


for node, cost in min_cost.items():
    for nbh, nbh_cost in get_nbhs_costs(node):
        if cost == min_cost[nbh] + nbh_cost:
            parents[node].append(nbh)

# Note: the nodes are (location, direction) pairs but we just want location
to_visit = set((end, dir) for dir in "<>^v" if min_cost[(end, dir)] == best_path_cost)
in_best_path = set()
while to_visit:
    node = to_visit.pop()
    in_best_path.add(node)
    to_visit.update(parents[node])

# print(parents)
# print(in_best_path)
# print(parents[((1, 139), '>')])
in_best_path = {point for point, dir in in_best_path}
print(len(in_best_path))


502


# Day 17

There's a computer register program basically. There's 3 registers: A, B, C. And there's 8 operations. The program is a sequence of numbers, which alternate between the operator ("opcode") and operand. Some operators have literal operands (numbers 0-7) and some have "Combo" operands, which means 0-3 are literals, 4 is A, 5 is B, 6 is C, and 7 is invalid/reserved.

Opcodes:

0: Divide A by 2^(Combo operand), truncate to integer, write to A
1: bitwise XOR register B with literal operand, write to B
2: Take Combo operand mod 8, write to B
3: If A is not 0, then jump to value of literal operand
4: bitwise XOR of register B and C, write to B
5: Outputs (Combo operand) mod 8
6: Divide A by 2^(Combo operand), truncate to integer, write to B
7: Divide A by 2^(Combo operand), truncate to integer, write to C


In [23]:
import re

with open("inputs/input17") as file:
    lines = file.readlines()
    lines = [line.strip() for line in lines]

A = int(lines[0][12:])
B = int(lines[1][12:])
C = int(lines[2][12:])

class State():
    def __init__(self, A, B, C):
        self.A = A
        self.B = B
        self.C = C
        self.p = 0 # pointer location
    def __str__(self):
        return f"A: {self.A}, B: {self.B}, C: {self.B}, p: {self.p}"

P = State(A, B , C)

instructions = [int(num) for num in re.findall(r"[0-9]+", lines[4])]

def op_0(operand):
    P.A = P.A // (2 ** combo_op(operand))

def op_1(operand):
    P.B = P.B ^ operand

def op_2(operand):
    P.B = combo_op(operand) % 8

def op_3(operand):
    if P.A != 0:
        P.p = operand - 2  # since we'll increment by 2

def op_4(operand):
    P.B = P.B ^ P.C

def op_5(operand):
    print(f"{combo_op(operand) % 8},", end="")

def op_6(operand):
    P.B = P.A // (2 ** combo_op(operand))

def op_7(operand):
    P.C = P.A // (2 ** combo_op(operand))

def combo_op(operand):
    match operand:
        case 4:
            return P.A
        case 5:
            return P.B
        case 6:
            return P.C
        case _: # < 4 or == 7
            return operand
    
while P.p < len(instructions):
    # print(P)
    instruction = instructions[P.p]
    operand = instructions[P.p + 1]
    locals()[f"op_{instruction}"](operand)
    P.p += 2


7,6,5,3,6,5,7,0,4,

Part 2: We want to initialize register A so that the program outputs the program itself.

Observations: The program ends in print, jump. So the instructions before it are what matter. Based on the actual instructions, the contents of B and C at the start of each loop do not matter. So A is the only state that matters.

So for every value of A, there is a number the program prints, and a new value of A. This captures all the relevant state.

When A = 0, 

In [7]:
import re
from collections import defaultdict

with open("inputs/input17") as file:
    lines = file.readlines()
    lines = [line.strip() for line in lines]

class State():
    def __init__(self, A, B, C):
        self.A = A
        self.B = B
        self.C = C
        self.p = 0 # pointer location
        self.out = []

    def __str__(self):
        return f"A: {self.A}, B: {self.B}, C: {self.B}, p: {self.p}"

instructions = [int(num) for num in re.findall(r"[0-9]+", lines[4])]

def op_0(operand, P):
    P.A = P.A // (2 ** combo_op(operand, P))

def op_1(operand, P):
    P.B = P.B ^ operand

def op_2(operand, P):
    P.B = combo_op(operand, P) % 8

def op_3(operand, P):
    if P.A != 0:
        P.p = operand - 2  # since we'll increment by 2

def op_4(operand, P):
    P.B = P.B ^ P.C

def op_5(operand, P):
    P.out.append(combo_op(operand, P) % 8)

def op_6(operand, P):
    P.B = P.A // (2 ** combo_op(operand, P))

def op_7(operand, P):
    P.C = P.A // (2 ** combo_op(operand, P))

def combo_op(operand, P):
    match operand:
        case 4:
            return P.A
        case 5:
            return P.B
        case 6:
            return P.C
        case _: # < 4 or == 7
            return operand
    
    
def simulate(instructions, reg_A):
    P = State(reg_A, 0, 0)
    while P.p < len(instructions):
        instruction = instructions[P.p]
        operand = instructions[P.p + 1]
        globals()[f"op_{instruction}"](operand, P)
        P.p += 2
    return P
    
A_map = {}

for A in range(128 * 8):
    P = simulate(instructions, A)
    A_map[A] = P.out[0]


A_map_inv = defaultdict(list)
for key, value in A_map.items():
    A_map_inv[value].append(key)

print(A_map_inv[0])

# Since A = A // 8 every time, we need a sequence of numbers
# which have acceptable values mod 1024 when doing // 8 division.

# And the last value has to terminate the program.
# Since the last value to print is 0, the only acceptable terminal
# value of A appears to be 5.

def next_possible_values(A_possible, next_out):
    # Will pass A={5} for the first thing, for example
    ...


print(instructions)


[5, 13, 21, 29, 33, 37, 45, 53, 54, 61, 62, 68, 69, 71, 76, 77, 79, 84, 85, 87, 92, 93, 95, 97, 100, 101, 108, 109, 116, 117, 124, 125, 161, 182, 190, 225, 289, 310, 318, 327, 335, 343, 351, 353, 417, 438, 446, 481, 545, 566, 574, 580, 583, 588, 591, 596, 599, 604, 607, 609, 612, 620, 628, 636, 673, 694, 702, 737, 801, 822, 830, 839, 847, 855, 863, 865, 929, 950, 958, 993]
[2, 4, 1, 2, 7, 5, 0, 3, 1, 7, 4, 1, 5, 5, 3, 0]


In [35]:
for i in range(8):
    i = i ^ 2
    print(2**i)

4
8
1
2
64
128
16
32
